# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [1]:
import numpy as np
import pandas as pd

# Challenge 1 - Index Functions

In this challenge, we will practice the advanced functions introduced in the lesson. However, before we start, we will load and evaluate our dataset.

In [2]:
admissions = pd.read_csv('../data/Admission_Predict.csv')

#Remove trailing spaces at the end of the column names
admissions.columns = admissions.columns.str.rstrip()

Let's evaluate the dataset by looking at the `head` function.

In [3]:
admissions.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90


Before beginning to work with this dataset and evaluating graduate admissions data, we will verify that there is no missing data in the dataset. Do this in the cell below.

In [4]:
#There isn't missing data
admissions.isna().sum()

Serial No.           0
GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
Chance of Admit      0
dtype: int64

Interestingly, there is a column that uniquely identifies the applicants. This column is the serial number column. Instead of having our own index, we should make this column our index. Do this in the cell below. Keep the column in the dataframe in addition to making it an index.

In [5]:
admissions.index = admissions["Serial No."]
admissions.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
Serial No.,,,,,,,,,
1,1,337,118,4,4.5,4.5,9.65,1,0.92
2,2,316,104,3,3.0,3.5,8.00,1,0.72
3,3,322,110,3,3.5,2.5,8.67,1,0.80
4,4,314,103,2,2.0,3.0,8.21,0,0.65
5,5,330,115,5,4.5,3.0,9.34,1,0.90


Turns out that `GRE Score` and `CGPA` also uniquely identify the data. Show this in the cell below.

In [6]:
GRE_unique_values = len(admissions["Serial No."].unique())
CGPA_unique_values = len(admissions["CGPA"].unique())
records = len(admissions)

print("GRE unique values:", GRE_unique_values)
print("CGPA unique values:", CGPA_unique_values)
print("Total records:", records)

#Some of CGPA values are repeated

GRE unique values: 385
CGPA unique values: 168
Total records: 385


Replace the index with an index comprised of two columns - `GRE Score` and `CGPA`. Remove the columns from the dataset as well. Make sure to use `inplace=True`.

Now change the index back to a sequence starting at zero using the `reset_index` function. Make sure to use `inplace=True`.

In [7]:
#Create new column "Comprised Score"
admissions["Comprised Score"] = admissions["GRE Score"].map(str) + "-" + admissions["CGPA"].map(str)

#Set'Comprised Score' as index
admissions.set_index("Comprised Score", inplace=True)

#Remove used columns (but they are needed next exercise!)
#admissions = admissions.drop(["GRE Score", "CGPA"], axis=1)

admissions.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
Comprised Score,,,,,,,,,
337-9.65,1,337,118,4,4.5,4.5,9.65,1,0.92
316-8.0,2,316,104,3,3.0,3.5,8.00,1,0.72
322-8.67,3,322,110,3,3.5,2.5,8.67,1,0.80
314-8.21,4,314,103,2,2.0,3.0,8.21,0,0.65
330-9.34,5,330,115,5,4.5,3.0,9.34,1,0.90


# Challenge 2 - Advanced Functions

In this part of the lab, we would like to test complex conditions on the entire dataset at once. Let's start by finding the number of rows where the CGPA is higher than 9 and the student has conducted research.

In [8]:
#Select rows with 'CGPA' higher than 9
CGPA_above_9 = admissions.loc[admissions["CGPA"] > 9]

#Select rows where 'Research' is 1
CGPA_above_9_and_research = CGPA_above_9.loc[CGPA_above_9["Research"] == 1]

CGPA_above_9_and_research.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
Comprised Score,,,,,,,,,
337-9.65,1,337,118,4,4.5,4.5,9.65,1,0.92
330-9.34,5,330,115,5,4.5,3.0,9.34,1,0.90
328-9.1,11,328,112,4,4.0,4.5,9.10,1,0.78
328-9.5,20,328,116,5,5.0,5.0,9.50,1,0.94
334-9.7,21,334,119,5,5.0,4.5,9.70,1,0.95


Now return all the rows where the CGPA is greater than 9 and the SOP score is less than 3.5. Find the mean chance of admit for these applicants.

In [9]:
CGPA_and_SOP_filtered = CGPA_above_9.loc[CGPA_above_9["SOP"] < 3.5]
CGPA_and_SOP_filtered["Chance of Admit"].mean()

0.8019999999999999

We would like to create a deciding factor column for each student. We standardize several columns and then pick the most important factor from a lookup table. If the standardized value is above 0.8, the student will be accepted.

We will start by creating a standardized column for `CGPA`, `GRE Score` and `LOR`. We will name these columns `CGPA_std`, `GRE_std`, and `LOR_std` respecively.

Recall that standardizing a column is done by subtracting the mean of the column from all observations in the column and then dividing each observation in the column by the column's standard deviation.

In the cell below, write a standardization function (a function that takes a column as input and returns a standardized column as output). Make sure to use the `numpy` versions of mean and standard deviation.

In [10]:
def standardize(col):
    # This function takes a column from a dataframe and returns a standardized column by subtracting the column's mean
    # and dividing by the column's standard deviation.
    
    col_mean = np.mean(col)
    col_stdev = np.std(col)
    
    col = (col-col_mean) / col_stdev
    
    return(col)

Now create the standardized columns `CGPA_std`, `GRE_std`, and `LOR_std` and add them to the `admissions` dataframe.

In [11]:
admissions["CGPA_std"] = standardize(admissions["CGPA"])
admissions["GRE_std"] = standardize(admissions["GRE Score"])
admissions["LOR_std"] = standardize(admissions["LOR"])

admissions.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std
Comprised Score,,,,,,,,,,,,
337-9.65,1,337,118,4,4.5,4.5,9.65,1,0.92,1.750174,1.755663,1.193197
316-8.0,2,316,104,3,3.0,3.5,8.00,1,0.72,-0.992501,-0.063450,0.076840
322-8.67,3,322,110,3,3.5,2.5,8.67,1,0.80,0.121191,0.456297,-1.039517
314-8.21,4,314,103,2,2.0,3.0,8.21,0,0.65,-0.643433,-0.236698,-0.481338
330-9.34,5,330,115,5,4.5,3.0,9.34,1,0.90,1.234884,1.149292,-0.481338


We will generate the decision choice at random using the code below. Please run the cell.

In [12]:
from random import choices
std_columns = ['CGPA_std', 'GRE_std', 'LOR_std']

decision_choice = choices(std_columns, k = admissions.shape[0])

Now create the deciding column using the `lookup` function. The lookup column is `decision_choice` found above. Call the column resulting from the lookup function `deciding_column` and add it to the `admissions` dataframe.

In [13]:
deciding_column = admissions.lookup(admissions.index, decision_choice) 
admissions["deciding_column"] = deciding_column

admissions.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std,deciding_column
Comprised Score,,,,,,,,,,,,,
337-9.65,1,337,118,4,4.5,4.5,9.65,1,0.92,1.750174,1.755663,1.193197,1.750174
316-8.0,2,316,104,3,3.0,3.5,8.00,1,0.72,-0.992501,-0.063450,0.076840,0.076840
322-8.67,3,322,110,3,3.5,2.5,8.67,1,0.80,0.121191,0.456297,-1.039517,-1.039517
314-8.21,4,314,103,2,2.0,3.0,8.21,0,0.65,-0.643433,-0.236698,-0.481338,-0.643433
330-9.34,5,330,115,5,4.5,3.0,9.34,1,0.90,1.234884,1.149292,-0.481338,-0.481338


Create a column called `decision` in the `admissions` dataframe. Assign 1 to this column if the value of `deciding_column` is greater than 0.8 and 0 otherwise.

In [14]:
admissions["decision"] = np.where(admissions["deciding_column"] > 0.8, 1, 0)
admissions.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std,deciding_column,decision
Comprised Score,,,,,,,,,,,,,,
337-9.65,1,337,118,4,4.5,4.5,9.65,1,0.92,1.750174,1.755663,1.193197,1.750174,1
316-8.0,2,316,104,3,3.0,3.5,8.00,1,0.72,-0.992501,-0.063450,0.076840,0.076840,0
322-8.67,3,322,110,3,3.5,2.5,8.67,1,0.80,0.121191,0.456297,-1.039517,-1.039517,0
314-8.21,4,314,103,2,2.0,3.0,8.21,0,0.65,-0.643433,-0.236698,-0.481338,-0.643433,0
330-9.34,5,330,115,5,4.5,3.0,9.34,1,0.90,1.234884,1.149292,-0.481338,-0.481338,0


How many applicants will be accepted to the program using the decision column? Compute the result below.

In [15]:
accepted = admissions.decision.sum()
print(accepted, "students will be accepted")

85 students will be accepted


# Challenge 3 - Method Chaining

To increase our coding efficiency, let's make a number of changes to our dataframe in one line of code.

In the cell below, remove all non character symbols from the column names, replace all spaces in column names to underscores, and change all upper case characters to lower case. Assign these new column values to `admissions.columns`.

In [16]:
#Replace spaces to underscores
admissions.columns = admissions.columns.str.replace(" ", "_")

#Remove all non character or '_' symbols
admissions.columns = admissions.columns.str.replace("[^a-zA-Z_]", "")

#Lower case
admissions.columns = map(str.lower, admissions.columns)

admissions.head()

,serial_no,gre_score,toefl_score,university_rating,sop,lor,cgpa,research,chance_of_admit,cgpa_std,gre_std,lor_std,deciding_column,decision
Comprised Score,,,,,,,,,,,,,,
337-9.65,1,337,118,4,4.5,4.5,9.65,1,0.92,1.750174,1.755663,1.193197,1.750174,1
316-8.0,2,316,104,3,3.0,3.5,8.00,1,0.72,-0.992501,-0.063450,0.076840,0.076840,0
322-8.67,3,322,110,3,3.5,2.5,8.67,1,0.80,0.121191,0.456297,-1.039517,-1.039517,0
314-8.21,4,314,103,2,2.0,3.0,8.21,0,0.65,-0.643433,-0.236698,-0.481338,-0.643433,0
330-9.34,5,330,115,5,4.5,3.0,9.34,1,0.90,1.234884,1.149292,-0.481338,-0.481338,0


In the cell below, give all student with a university rating of 4 or higher a 10 point boost on their GRE score and split the column into 4 bins using the `cut` function. Assign this new score to the variable `adjusted_gre`.

In [17]:
#Sum the 10 points boost
admissions["gre_score"] = np.where(admissions["university_rating"] > 4, admissions["gre_score"]+10, admissions["gre_score"])

#Split into 4 bins
admissions["adjusted_gre"] = pd.cut(admissions["gre_score"], 4)

admissions.head()

,serial_no,gre_score,toefl_score,university_rating,sop,lor,cgpa,research,chance_of_admit,cgpa_std,gre_std,lor_std,deciding_column,decision,adjusted_gre
Comprised Score,,,,,,,,,,,,,,,
337-9.65,1,337,118,4,4.5,4.5,9.65,1,0.92,1.750174,1.755663,1.193197,1.750174,1,"(335.0, 350.0]"
316-8.0,2,316,104,3,3.0,3.5,8.00,1,0.72,-0.992501,-0.063450,0.076840,0.076840,0,"(305.0, 320.0]"
322-8.67,3,322,110,3,3.5,2.5,8.67,1,0.80,0.121191,0.456297,-1.039517,-1.039517,0,"(320.0, 335.0]"
314-8.21,4,314,103,2,2.0,3.0,8.21,0,0.65,-0.643433,-0.236698,-0.481338,-0.643433,0,"(305.0, 320.0]"
330-9.34,5,340,115,5,4.5,3.0,9.34,1,0.90,1.234884,1.149292,-0.481338,-0.481338,0,"(335.0, 350.0]"
